In [ ]:
# !pip install scikit-learn
# !pip install numpy==1.16.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ",len(tf.config.experimental.list_physical_devices('GPU')))

2.4.1
/device:GPU:0
Num GPUs Available:  1


In [ ]:
# from tensorflow.keras.callbacks import Callback

In [ ]:
# # Included this
# def custom_stopping(value=0.5, verbose=0):
#     early = EarlyStoppingByLossVal(monitor='val_loss', value=value, verbose=verbose)
#     return early

# class EarlyStoppingByLossVal(Callback):
#     def __init__(self, monitor='val_acc', value=0.95, verbose=0):
#         super(Callback, self).__init__()
#         self.monitor = monitor
#         self.value = value
#         self.verbose = verbose

#     def on_epoch_end(self, epoch, logs={}):
#         current = logs.get(self.monitor)
#         # if current is None:
#         # warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

#         if current < self.value:
#             if self.verbose > 0:
#                 print("Epoch %05d: early stopping THR" % epoch)
#             self.model.stop_training = True

In [ ]:
def DataPreparation(data_input_file):
  #print('Bevilacqua  et al. 2015 {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  
  Y = data['y']
  folds = data['folds']
  
  return X,Y,folds



In [ ]:
# X,Y,f=DataPreparation('/content/drive/My Drive/SNOW/MHEALTH.npz')
# print(X.shape)
# print(Y.shape)

In [ ]:
# print(f[0])
# print(f[0][])
# print(f[0][0])

In [ ]:
def build_model_1(row, col, num_classes):
  layers = [
            tf.keras.layers.Conv2D(filters=10,kernel_size=(3,5), activation=tf.nn.relu),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=1),

            tf.keras.layers.Conv2D(filters=2, kernel_size=(2,4), activation=tf.nn.relu),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=1),

            tf.keras.layers.Conv2D(filters=2, kernel_size=(2,2), activation=tf.nn.relu),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPool2D(pool_size=(3,2), strides=1),

            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=500, activation=tf.nn.relu),
            tf.keras.layers.Dropout(rate=0.5),
            tf.keras.layers.Dense(units=250, activation=tf.nn.relu),
            tf.keras.layers.Dropout(rate=0.5),
            tf.keras.layers.Dense(units=125, activation=tf.nn.relu),
            tf.keras.layers.Dropout(rate=0.5),

            tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax)
  ]
  model = tf.keras.Sequential(layers)
  return model


In [ ]:
def Train(X,Y,folds):
  avg_acc = []
  avg_recall = []
  avg_f1 = []
  n_class = Y.shape[1]
  _, _, rows, cols = X.shape

  for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]
    X_train = X[train_idx]
    Y_train = Y[train_idx]
    X_test = X[test_idx]
    Y_test = Y[test_idx]

    model = build_model_1(rows, cols, n_class)

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='Adam')
    model.fit(X_train, Y_train, batch_size=128, epochs=300)
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(Y_test, axis=1)
    acc_fold = accuracy_score(y_true, y_pred)
    avg_acc.append(acc_fold)
    recall_fold = recall_score(y_true, y_pred, average='macro')
    avg_recall.append(recall_fold)
    f1_fold = f1_score(y_true, y_pred, average='macro')
    avg_f1.append(f1_fold)
    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
    print('________________________________________________________________')
    
  return avg_acc, avg_recall,avg_f1


In [ ]:
def ReportAccuracies(avg_acc, avg_recall,avg_f1):
  ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
  ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
  ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
  print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
  print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
  print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))

In [ ]:
def RunTheModel(input_file):
  X,Y,f=DataPreparation(input_file)
  avg_acc, avg_recall,avg_f1 = Train(X,Y,f)
  ReportAccuracies(avg_acc, avg_recall,avg_f1)

  

In [ ]:
tf.keras.backend.set_image_data_format('channels_first')
# tf.keras.backend.set_image_data_format('channels_first')

In [ ]:
RunTheModel('/content/drive/My Drive/SNOW/WISDM.npz')
# RunTheModelOpt('/content/drive/My Drive/Opportunity/Opportunity_train_X.npz', '/content/drive/My Drive/Opportunity/Opportunity_train_y.npz', '/content/drive/My Drive/Opportunity/Opportunity_test_X.npz', '/content/drive/My Drive/Opportunity/Opportunity_test_Y.npz')


Epoch 1/300


ValueError: ignored